<img src="images/Picture0.png" width=200x />

# Notebook 03 - Sensitivity Analysis

### Covered in this notebook:
* Introduction to sensitivity analysis:
    * Dual price or shadow price
    * Reduced cost
    * Slack

### Credits:
* [Shadow price and reduced cost](https://www.or-as.be/blog/tsbd_sp), article by Mario Vanhoucke for OR-AS
* [Optimization course notes](https://theory.stanford.edu/~trevisan/cs261/lecture06.pdf) by Luca Trevisan at Stanford University
* Gurobi support and resources

In [1]:
import gurobipy as gp
from gurobipy import GRB

## Sensitivity Analysis

Let's take a moment to discuss an important topic to LP problems.  From a human standpoint, finding the solution to our optimization problem is often only the beginning in terms of practical solutions.  We will still have much to discuss sorting out particulars.  What if our material costs are too high for shareholders?  What if a supplier's availability changes?  What if a material we would like to include was deemed too inefficient for optimality?  We will want to know what happens to our solution if we adjust our inputs, and it is useful not to have to run our model again and again to figure these things out.

<strong>Sensitivity analysis</strong> measures the solution's response to changes in the problem: how "sensitive" our solution is.  Gurobi provides us with quite a bit of additional information about our problem; let's discuss some of the relevant pieces.

### Dual Price

The <strong>dual price</strong> or <strong>shadow price</strong> of a constraint refers to the change in the objective value for each unit change in the constraint's right-hand side.  (If we are optimizing $f()$ subject to $g()>=c$, the shadow price gives the change in $f()$ when $c$ is increased by $1$.)  We retrieve shadow price by querying the `Pi` attribute of our constraint; we can query the individual restraint if we gave it a name or assigned it to a variable, or we can retrieve the shadow prices of all constraints in our model in the order they were added by calling `getAttr('Pi')` on the model itself.

<i>(Note for interested individuals: Gurobi computes and can deliver the [dual](https://theory.stanford.edu/~trevisan/cs261/lecture06.pdf) of your program by writing the model to a DLP or DUA file.  More about reading/writing models in Notebook 05.)</i>

### Reduced cost

Suppose our optimal solution occurs when one or more of our variables takes a zero value.  The <strong>reduced cost</strong> informs us of the amount by which the variable's objective function parameter would need to improve for the variable to take a positive value in the optimal solution.

The reduced cost is equivalent to the shadow price of the variable's nonnegativity constraint; that is, if we are optimizing $f()$ subject to $x>= 0$, then the reduced cost gives the change in $f()$ when the constraint is changed to $x>=1$.  If we require $x$ to be present in our optimal solution, how much does this cost our optimality?  And since our problem is linear, we can also think of this as the amount by which the coefficient of $x$ would need to improve (increase or decrease, depending on our objective sense) for $x$ to appear in our solution, to be "cheaper" than the variables which are present already.  The reduced cost of a variable is given by its `RC` attribute.

### Slack

A <strong>binding</strong> constraint is one which is involved in our optimal solution; changing the right-hand side of such a constraint would change the value of the optimal solution.  Not all constraints are necessarily binding, and nonbinding constraints have some <strong>slack</strong>: defined as the difference between the actual usage or production and the imposed limits on usage or production, slack tells us how much we could increase or decrease the constraint's right-hand side before impacting our optimal solution.  It tells us what is "left over" after applying the constraint.  Naturally, slack is zero for binding constraints and nonzero for nonbinding constraints.  A constraint's slack is given by its `Slack` attribute.

Let's analyze the following problem under a sensitivity analysis.

## Exercise: Power Grid

You manage the electric company Midwest PG.  The grid draws power from five sources: a nuclear plant, a coal plant, a solar plant, a wind farm, and a hydroelectric dam.  Power flows into three hubs before it is then routed to its end destination.  Each source has a limited supply of power, while each hub has a minimum demand.  Not every source is connected to every hub, and there is a cost to delivering energy which is based on the length of the power line.  The system is depicted below:

<img src="images/N03_images/powerGrid.png">

Your team and your shareholders have some questions about your operation.  Solve the system for the optimal solution, then use that information to respond to the questions.

In [2]:
# instantiate model
m = gp.Model("Power Grid")

Set parameter Username


In [3]:
# add variables to represent power from each source to each hub
n1 = m.addVar(obj=4, name='n1')
n2 = m.addVar(obj=5, name='n2')
c1 = m.addVar(obj=3, name='c1')
c2 = m.addVar(obj=2, name='c2')
s1 = m.addVar(obj=2, name='s1')
s3 = m.addVar(obj=6, name='s3')
w2 = m.addVar(obj=4, name='w2')
w3 = m.addVar(obj=3, name='w3')

In [4]:
# add constraints:
# sources limited
m.addConstr(n1 + n2 == 1500, name='nuclear')
m.addConstr(c1 + c2 == 3000, name='coal')
m.addConstr(s1 + s3 == 2400, name='solar')
m.addConstr(w2 + w3 == 1800, name='wind')

<gurobi.Constr *Awaiting Model Update*>

In [5]:
# add constraints:
# hubs must meet demand
m.addConstr(n1 + c1 + s1 >= 2400, name='hub1')
m.addConstr(n2 + c2 + w2 >= 1600, name='hub2')
m.addConstr(s3 + w3 >= 2000, name='hub3') ##

<gurobi.Constr *Awaiting Model Update*>

In [6]:
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x12bbdc75
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 3e+03]
Presolve removed 4 rows and 4 columns
Presolve time: 0.01s
Presolved: 3 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2991196e+04   2.751250e-01   0.000000e+00      0s
       1    2.3000000e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.300000000e+04


In [7]:
for v in m.getVars():
    print(v.varName, '=', v.x)

print()

for v in m.getVars():
    print(v.varName, 'reduced cost =', v.RC)

print()

for c in m.getConstrs():
    print(c.constrName, 'slack =', c.Slack)

print()

for c in m.getConstrs():
    print(c.constrName, 'shadow price =', c.Pi)

print()

print('total expenses:', m.objVal)

n1 = 1500.0
n2 = 0.0
c1 = 0.0
c2 = 3000.0
s1 = 2200.0
s3 = 200.0
w2 = 0.0
w3 = 1800.0

n1 reduced cost = 0.0
n2 reduced cost = 1.0
c1 reduced cost = 1.0
c2 reduced cost = 0.0
s1 reduced cost = 0.0
s3 reduced cost = 0.0
w2 reduced cost = 5.0
w3 reduced cost = 0.0

nuclear slack = 0.0
coal slack = 0.0
solar slack = 0.0
wind slack = 0.0
hub1 slack = -1300.0
hub2 slack = -1400.0
hub3 slack = 0.0

nuclear shadow price = 4.0
coal shadow price = 2.0
solar shadow price = 2.0
wind shadow price = -1.0
hub1 shadow price = 0.0
hub2 shadow price = 0.0
hub3 shadow price = 4.0

total expenses: 23000.0


### Questions:

#### Shadow price
1. Which of our hubs is the most expensive to serve?
2. Which energy source(s) should Midwest PG invest in to decrease expenses in the future?
3. Historically, Hub 2 has seen an increase in demand of 400 units during football weekends.  What does this increase in demand cost us?

#### Reduced cost
1. Midwest PG wants to upgrade lines to bring down their carriage costs.  Which lines most need to be improved?
2. What would we need to bring their carriage costs down to in order to make those lines competitive?
3. Are there any lines we need to close outright?

#### Slack
1. Are we free to reduce production at any of our locations?
2. If Midwest PG were considering serving a new hub, could we do so on our existing infrastructure?

### Answers:
1. Hub 3
2. Wind
3. 800 currency

1. n1, n2, w2
2. 3, 2, -2
3. w2

1. nuclear and coal
2. yes